# Some bonus plots for illustrating stuff for A_FISH

In [ ]:
%matplotlib widget
from pathlib import Path
from shutil import copy

import flammkuchen as fl
import lotr.plotting as pltltr
import numpy as np
import pandas as pd
from lotr import A_FISH, LotrExperiment, dataset_folders
from lotr.utils import zscore

COLS = pltltr.COLS
from matplotlib import pyplot as plt
from tqdm import tqdm

### Contour masks for a fish

In [ ]:
exp = LotrExperiment(A_FISH)

rois = exp.rois_stack
ring_rois = exp.color_rois_by(np.ones(exp.n_hdns))
proj = ring_rois.max(0)[:, :, 0]

In [ ]:
plt.figure(figsize=(3, 3), facecolor="none")
plt.contour(proj, levels=1, colors=[COLS["ring"]], linewidths=1)
plt.axis("equal")
plt.axis("off")
pltltr.savefig("rois_mask")

In [ ]:
np.random.choice(np.argwhere(exp.nonhdn_indexes)[:, 0], 5)

In [ ]:
np.random.choice(exp.hdn_indexes, 5)

In [ ]:
idxs = [480, 44, 598, 363, 285] + [131, 503, 657, 188, 34]
n = len(idxs) // 2  # number of neurons for each type
s = 5
lw = 0.5
t_slice_s = (0, 1900)

t_slice = slice(*[t * exp.fs for t in t_slice_s])

f, ax = plt.subplots(figsize=(3, 1.5))
plt.plot(
    exp.time_arr,
    zscore(exp.traces[:, idxs[:n]]) + np.arange(n) * s,
    lw=lw,
    c=COLS["ring"],
)
plt.plot(
    exp.time_arr,
    zscore(exp.traces[:, idxs[n:]]) + np.arange(n) * s + n * s,
    lw=lw,
    c=".3",
)

pltltr.add_scalebar(
    ax, xunits="s", xlen=200, ylen=10, ylabel="ΔF z sc.", lw=0.5, fontsize=5
)
plt.show()

pltltr.savefig("traces")

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pcs = PCA(n_components=2).fit_transform(exp.traces[:, exp.hdn_indexes])
pcs_t = PCA(n_components=2).fit_transform(exp.traces[:, exp.hdn_indexes].T)

In [ ]:
f, axs = plt.subplots(
    1, 2, figsize=(5, 2), gridspec_kw=dict(left=0.3, bottom=0.2, wspace=0.5)
)
axs[0].plot(pcs[:, 0], pcs[:, 1], lw=1, c=COLS["ring"])
axs[1].scatter(pcs_t[:, 0], pcs_t[:, 1], s=10, lw=0, color=COLS["ring"])
for ax in axs:
    ax.set(xlabel="PC1", ylabel="PC2")
    ax.axis("equal")
    pltltr.despine(ax)
axs[1].set(yticks=[-100, 0, 100], xticks=[-100, 0, 100], xlim=(-110, 110))
pltltr.savefig("pcs")

### Phase-sorted traces

In [ ]:
sorted_traces = exp.traces[:, exp.hdn_indexes][:, np.argsort(exp.rpc_angles)]
phase = exp.network_phase
phase[1:][np.abs(np.diff(phase)) > np.pi] = np.nan
phases = exp.rpc_angles[np.argsort(exp.rpc_angles), np.newaxis]

In [ ]:
t_slice_s = (0, 1000)

t_slice = slice(*[t * exp.fs for t in t_slice_s])
f, axs = plt.subplots(
    1,
    2,
    figsize=(5, 2),
    gridspec_kw=dict(left=0.2, bottom=0.2, width_ratios=(0.03, 1), wspace=0),
)

ax = axs[1]
im = ax.imshow(
    sorted_traces[t_slice, :].T,
    extent=[0, t_slice_s[1], 0, exp.n_hdns],
    aspect="auto",
    cmap=COLS["dff_plot"],
    vmin=-1.7,
    vmax=2.0,
)
ax.plot(
    exp.time_arr[t_slice],
    (phase[t_slice] / np.pi + 1) * exp.n_hdns / 2,
    c=COLS["ph_plot"],
    lw=1,
    label="Network phase",
)
ax.legend(loc=2, bbox_to_anchor=(0.71, 1.15), fontsize=7)
ax.set(xlabel="Time (s)")
pltltr.add_dff_cbar(
    im,
    ax,
    (1.05, 0.04, 0.02, 0.15),
    title="ΔF (Z.)",
    titlesize=6,
    labelsize=5,
    ticklabels=None,
)
pltltr.despine(ax, ["left", "right", "top"])

axs[0].set(ylabel="ROI n. (phase sorted)")
axs[0].imshow(phases, cmap=COLS["phase"], aspect="auto")
pltltr.despine(axs[0], ["bottom", "right", "top"])

pltltr.savefig("traces_by_phase")

In [ ]:
plt.figure()
plt.imshow(phases, cmap=COLS["phase"])